In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.optim import lr_scheduler
import torch.nn.functional as F
import time
from options.test_options import TestOptions, Opt_test
from data import DataLoader
from models import create_model
from util.writer import Writer
from test import run_test
from util.mesh_viewer import *


In [2]:
opt = Opt_test()
opt.parse()
# These options must match the options for training!
opt.dataroot = 'datasets/human_seg' 
opt.name = 'human_seg'
opt.arch = 'meshunet'
opt.dataset_mode  = 'segmentation'
opt.ncf = [32, 64, 128, 256] 
opt.pool_res = [1800, 1350, 600]
opt.ninput_edges = 2280
opt.norm = 'batch'
opt.resblocks = 3 
opt.batch_size = 12
opt.export_folder = 'humseg_test'

dataset = DataLoader(opt)
dataset_size = len(dataset)
print('#test meshes = %d' % dataset_size)
model = create_model(opt)
writer = Writer(opt)

def test_all(opt, dataset, model, writer, epoch=-1):
    print('Running Test')
    #opt = TestOptions().parse()
    opt.serial_batches = True  # no shuffle
    #dataset = DataLoader(opt)
    #model = create_model(opt)
    
    # test
    writer.reset_counter()
    for i, data in enumerate(dataset):
        model.set_input(data)
        ncorrect, nexamples = model.test()
        writer.update_counter(ncorrect, nexamples)
    writer.print_acc(epoch, writer.acc)
    return writer.acc

def test_random(opt, dataset, model):
    num_obj = np.random.randint(len(dataset))
    opt.serial_batches = True 
    for i, data in enumerate(dataset):
            if i == num_obj//16:
                idx = num_obj - num_obj//16*16
                model.set_input(data)
                out = model.forward()
                pred_class = out.data.max(1)[1]
                #ncorrect, nexamples = model.test()
                print('Correct class: %s' % dataset.dataset.classes[data['label'][idx]])
                print('Predicted class: %s' % dataset.dataset.classes[pred_class[idx]])
        
                obj_name = dataset.dataset.paths[num_obj][0]
                print('see object file %s for visualization' % obj_name.split('\\')[-1])
            

loaded mean / std from cache
#test meshes = 18
loading the model from ./checkpoints\human_seg\latest_net.pth


In [3]:
 test_all(opt, dataset, model, writer, epoch=-1)

Running Test
epoch: -1, TEST ACC: [92.385 %]



0.9238476223415799

In [4]:
test_random(opt, dataset, model)

IndexError: arrays used as indices must be of integer (or boolean) type

In [5]:
num_obj = np.random.randint(len(dataset))
opt.serial_batches = True 
for i, data in enumerate(dataset):
        #if i == num_obj//opt.batch_size:
        idx = num_obj - num_obj//opt.batch_size*opt.batch_size
        model.set_input(data)
        out = model.forward()
        pred_class = out.data.max(1)[1]
        #ncorrect, nexamples = model.test()
        print(data['soft_label'].shape)
        print(data['label'].shape)
        print(data['mesh'][0].edges)
        print('Correct class: %s' % dataset.dataset.classes[data['label'][idx]])
        print('Predicted class: %s' % dataset.dataset.classes[pred_class[idx]])

        obj_name = dataset.dataset.paths[num_obj][0]
        print('see object file %s for visualization' % obj_name.split('\\')[-1])

(12, 2280, 8)
(12, 2280)
[[334 456]
 [  2 456]
 [  2 334]
 ...
 [ 98 744]
 [ 79 744]
 [ 79 600]]


IndexError: arrays used as indices must be of integer (or boolean) type

In [6]:
print(data['soft_label'][0].shape)
print(data['label'][0].shape)
print(data['mesh'][0].edges_count)
print(data['mesh'][0].gemm_edges.shape)
obj_name = dataset.dataset.paths[i]
print('see object file %s for visualization' % obj_name.split('\\')[-1])

(2280, 8)
(2280,)
2250
(2250, 4)
see object file shrec__1.obj for visualization


In [8]:
data['mesh'][0].edges.shape

(600, 2)